In [ ]:
# 데이터처리를 위한 패키지
import pandas as pd
import numpy as np

# 날짜 처리를 위한 패키지
import datetime as dt

# 파일 위치 이름 가져오기 위한 패키지
import glob

# encoding 문제 해결을 위한 패키지
import chardet

In [ ]:
# air 데이터 병합
air_2016_seoul = pd.read_csv('D:/project/contest/data/processed/air_2016_seoul.csv')
air_2017_seoul = pd.read_csv('D:/project/contest/data/processed/air_2017_seoul.csv')
air_2018_seoul = pd.read_csv('D:/project/contest/data/processed/air_2018_seoul.csv')
air = pd.concat([air_2016_seoul,air_2017_seoul,air_2018_seoul], ignore_index=True)
air.to_csv('D:/project/contest/data/processed/p_air.csv', encoding='utf-8')

In [ ]:
weather = pd.read_csv('d:/project/contest/data/processed/p_weather.csv', parse_dates=['date'])
uv = pd.read_csv('D:/project/contest/data/processed/p_uv.csv', parse_dates=['date'])

# 서울시만 추출
w_seoul = weather.loc[weather['loc']==108] # 서울(108)
uv_seoul = uv.loc[uv['1단계']=='서울특별시']

uv_seoul.columns = ['date', '지점번호', 'pvn_nm', 'bor_nm', 'uv', 'uv_after1', 'uv_after2']

# 시단위 평균으로 병합
uv_seoul = uv_seoul[['date', 'pvn_nm', 'uv']].groupby(['date', 'pvn_nm']).mean().reset_index()
air_seoul = air.groupby(['date']).mean().reset_index()

# 자외선지수 병합
w_uv = pd.merge(weather, uv_seoul[['date','uv']], on='date',how='left')

# 공기오염물질 병합
w_uv_a = pd.merge(w_uv, air_seoul, on='date',how='left')

# 서울만 중간저장
w_uv_a.loc[w_uv_a['loc']==108].to_csv('D:/project/contest/data/processed/p_wUVair_seoul.csv', encoding='utf-8')

In [ ]:
# 중간저장파일 불러오기
# w_seoul = pd.read_csv('D:/project/contest/data/processed/p_wUVair_seoul.csv', index_col=0)
w_seoul = w_uv_a.copy()

# 전후의 평균치로 대체
air_pollution = ['SO2', 'CO', 'O3', 'NO2', 'PM10', 'PM25']
to_fillNaN = dict(w_seoul[air_pollution].loc[(w_seoul.date=='2017-12-30')|(w_seoul.date=='2018-01-01')].mean())
w_seoul_airpol = w_seoul.iloc[:,-6:]
w_seoul_airpol = w_seoul_airpol.fillna(value=to_fillNaN, limit=1)
w_seoul[air_pollution] = w_seoul_airpol
w_seoul.to_csv('D:/project/contest/data/processed/p_wUVair_seoul.csv', encoding='utf-8')